In [1]:
import os
import pandas as pd
import numpy as np

# basic solution
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import KFold

# creative solution
from numpy import vstack
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
def weighted_accuracy(pred, true):
    assert(len(pred) == len(true))
    num_labels = len(true)
    num_pos = sum(true)
    num_neg = num_labels - num_pos
    frac_pos = num_pos/num_labels
    weight_pos = 1/frac_pos
    weight_neg = 1/(1-frac_pos)
    num_pos_correct = 0
    num_neg_correct = 0
    for pred_i, true_i in zip(pred, true):
        num_pos_correct += (pred_i == true_i and true_i == 1)
        num_neg_correct += (pred_i == true_i and true_i == 0)
    weighted_accuracy = ((weight_pos * num_pos_correct) 
                         + (weight_neg * num_neg_correct))/((weight_pos * num_pos) + (weight_neg * num_neg))
    return weighted_accuracy

### Preprocessing and Feature Extraction:</h3><p>

In [3]:
# reading the csv files
df_train = pd.read_csv("train_2016.csv", sep=',', encoding='unicode_escape', thousands=',')
df_test = pd.read_csv("test_2016_no_label.csv", sep=',', encoding='unicode_escape', thousands=',')
del df_train['County']
del df_test['County']

# get true labels
n,d = df_train.shape
y = np.ones(n)
y[df_train.DEM < df_train.GOP] = 0

# delete unneeded columns
del df_train['DEM']
del df_train['GOP']

# scale the data
scaler = StandardScaler()
scaler.fit(df_train.values)
X_training = scaler.transform(df_train.values)
X_testing = scaler.transform(df_test.values)

In [4]:
print("The size of X_training is:", X_training.shape)
print("The size of X_testing is:", X_testing.shape)

The size of X_training is: (1555, 7)
The size of X_testing is: (1555, 7)


## Part 1: Basic Machine Learning Models

### 1.1 Use Three Training Algorithms

In [5]:
models = {
    "knn_3uni": KNeighborsClassifier(n_neighbors=3, weights='uniform'),
    "knn_3dis": KNeighborsClassifier(n_neighbors=3, weights='distance'),
    "knn_5uni": KNeighborsClassifier(n_neighbors=5, weights='uniform'),
    "knn_5dis": KNeighborsClassifier(n_neighbors=5, weights='distance'),
    "knn_9uni": KNeighborsClassifier(n_neighbors=9, weights='uniform'),
    "knn_9dis": KNeighborsClassifier(n_neighbors=9, weights='distance'),
    "knn_13uni": KNeighborsClassifier(n_neighbors=13, weights='uniform'),
    "knn_13dis": KNeighborsClassifier(n_neighbors=13, weights='distance'),
    
    "lda_svd": LinearDiscriminantAnalysis(solver='svd'),
    "lda_lsqr": LinearDiscriminantAnalysis(solver='lsqr'),
    "lda_eigen": LinearDiscriminantAnalysis(solver='eigen'),

    "svc_1lin": SVC(kernel='linear', C=1),
    "svc_10lin": SVC(kernel='linear', C=10),
    "svc_100lin": SVC(kernel='linear', C=100),
    "svc_1rbf": SVC(kernel='rbf', C=1),
    "svc_10rbf": SVC(kernel='rbf', C=10),
    "svc_100rbf": SVC(kernel='rbf', C=100),
    "svc_1poly": SVC(kernel='poly', C=1),
    "svc_10poly": SVC(kernel='poly', C=10),
    "svc_100poly": SVC(kernel='poly', C=100)
}

### 1.2 Training and Validation
Split the data to a training set and validation set or performing a cross-validation for model selection.

In [6]:
# set up dictionary to hold the cross-validation scores for each model
scores = {}
for m in models.keys(): 
    scores[m] = []

# cross-validation on 10 folds
kf10 = KFold(n_splits=10,shuffle=True)
for train_index, test_index in kf10.split(X_training):
    X_train, X_test = X_training[train_index], X_training[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for name, model in models.items():
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        scores[name].append(weighted_accuracy(predictions, y_test))

# print the mean score for each model
for n, s in scores.items(): 
    print(n,':',np.round(np.mean(np.array(s)),3))

knn_3uni : 0.692
knn_3dis : 0.694
knn_5uni : 0.684
knn_5dis : 0.691
knn_9uni : 0.669
knn_9dis : 0.674
knn_13uni : 0.661
knn_13dis : 0.674
lda_svd : 0.69
lda_lsqr : 0.69
lda_eigen : 0.69
svc_1lin : 0.642
svc_10lin : 0.645
svc_100lin : 0.645
svc_1rbf : 0.681
svc_10rbf : 0.718
svc_100rbf : 0.715
svc_1poly : 0.659
svc_10poly : 0.699
svc_100poly : 0.707


### 1.3 Comments

k-nearest neighbors (k-NN), linear discriminant analysis (LDA), and support vector machine (SVM) learning methods were chosen because they seemed suitable for a binary classification where the features aren't discrete. Each learning method has potentioal strengths and weaknesses. k-NN doesn't depend on the data being separable like the other two but it can be sensitive to outliers. LDA assumes that the data forms a spherical normal distribution but it takes the entire feature matrix into account so it might be better than the SVM in that aspect. Meanwhile the SVM might be better than the LDA in that it only depends on the support vectors and the use of kernels means the separation of different classes doesn't have to be linear unlike for LDA.

For the k-NN classifier, a range of neighbors from 3 to 13 were selected, and the neighbors were weighted uniformly or by distance. For LDA, different solvers were tested. For the SVM, the linear, rbf, and polynomial kernels were tested, as well as a range of 1 to 100 for C value. Models were compared using 10-fold cross validation.

## Part 2: Artifical Neural Network

In [7]:
class CSVDataset(Dataset):
    def __init__(self, csv_file, mode = 'train'):
        self.mode = mode
        df = pd.read_csv(csv_file, sep=',',encoding='unicode_escape',thousands=',')
        del df['County']
        
        if self.mode == 'train':
            # labels        
            n,d = df.shape
            y = np.ones(n)
            y[df.DEM < df.GOP] = 0
            del df['DEM']
            del df['GOP']
            del df['FIPS']
            
            scaler = StandardScaler()
            scaler.fit(df.values)

            self.X = scaler.transform(df.values)
            self.y = y.reshape(n,1)
        else:
            del df['FIPS']
            scaler = StandardScaler()
            scaler.fit(df.values)
            
            self.X = scaler.transform(df.values)
                
        
    def __len__(self):
        # Returns count of samples (an integer) 
#         raise NotImplementedError
        return len(self.X)

    def __getitem__(self, idx):
        # Given an index, returns the correponding datapoint. 
        # This function is called from dataloader like this:
        # img, label = CSVDataset.__getitem__(99)  # For 99th item
#         raise NotImplementedError
        # labels
        
        if self.mode == 'train':
            X = torch.Tensor(self.X[idx]).float()
            y = torch.Tensor(self.y[idx]).float()
            return X,y
        else:
            X = torch.Tensor(self.X[idx]).float()
            return X
    
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
        
        

In [8]:
class Net(nn.Module):
    def __init__(self,dim):
        """
        initialize most NN layers here.
        """
        super(Net, self).__init__()
        # an affine operation: y = Wx + b

        self.fc1 = nn.Linear(dim, 160)
        self.b1 = nn.BatchNorm1d(160)
        self.fc2 = nn.Linear(160, 64)
        self.b2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 8)
        self.b3 = nn.BatchNorm1d(8)
        self.fc4 = nn.Linear(8, 1)

    def forward(self, x):
        """
        Define in what order the input x is forwarded through all the NN layers to become a final output. 
        """        
        x = F.relu(self.fc1(x))
        x = self.b1(x)
        x = F.relu(self.fc2(x))
        x = self.b2(x)
        x = F.relu(self.fc3(x))
        x = self.b3(x)
        x = torch.sigmoid(self.fc4(x))
        return x

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

def train(csv_file,dim):
    # Initialize an object of the model class
    net = Net(dim).to(device)
    # Define loss function
    criterion = nn.MSELoss()
    # Create optimizer
    # optimizer = optim.SGD(net.parameters(), lr=0.01)
    optimizer = optim.Adam(net.parameters())
    # Initialize an object of the dataset class
    dataset = CSVDataset(csv_file)
    train, test = dataset.get_splits()
    # Wrap a dataloader around the dataset object.
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=311, shuffle=True)
    # Beging training!
    for batch_idx, (input, target) in enumerate(train_dl):
        # Always want to use zero_grad(), backward(), and step() in the following order.
        # zero_grad clears old gradients from the last step (otherwise you’d just accumulate the gradients from all loss.backward() calls).
        optimizer.zero_grad()
        # As said before, only code as below if the network belongs to the nn.Module class.
        input = input.view(-1,dim)
        output = net(input)        
        loss = criterion(output, target)        
        # loss.backward() computes the derivative of the loss w.r.t. the parameters (or anything requiring gradients) using backpropagation.
        loss.backward()
        # optimizer.step() causes the optimizer to take a step based on the gradients of the parameters.
        optimizer.step()
    
    #validate 
    predictions, actuals = list(), list()
    for batch_idx, (input, target) in enumerate(test_dl):
        input = input.view(-1,dim)
        output = net(input)
        
        yhat = torch.round(output)
#         yhat = output
    
        yhat = yhat.detach().numpy()
        
        actual = target.numpy()
        actual = actual.reshape((len(actual), 1))
        
        predictions.append(yhat)
        actuals.append(actual)
        
    predictions, actuals = vstack(predictions), vstack(actuals)
    
#     # accuracy
    acc = weighted_accuracy(predictions, actuals)    
        
    return loss, predictions, acc, net

### 2.1 New features using train_2012

In [10]:
df_2016 = pd.read_csv("train_2016.csv", sep=',',encoding='unicode_escape',thousands=',')
df_2012 = pd.read_csv("train_2012.csv", sep=',',encoding='unicode_escape',thousands=',')

df_mean0 = (df_2016.iloc[:,2:]+df_2012.iloc[:,2:])/2
df_mean = df_2016.iloc[:,:2].join(df_mean0)
df_mean.to_csv("train_mean.csv",index=False)

### 2.2 New features using graph.csv

In [11]:
df_graph = pd.read_csv("graph.csv", sep=',',encoding='unicode_escape',thousands=',')
df_train2016 = pd.read_csv("train_2016.csv", sep=',',encoding='unicode_escape',thousands=',')
n,d = df_graph.shape
for i in range(n):
    index_2 = df_train2016.FIPS.values == df_graph.DST.iloc[i]        
    if any(index_2): 
        df_graph.loc[i,'DST_C'] = df_train2016.DEM.values[index_2]
    else:
        df_graph.loc[i,'DST_C'] = np.nan
    
df_graph.dropna(axis = 0, how = 'any', inplace = True)

In [12]:
# new df_graph
n,d = df_graph.shape

df1 = pd.DataFrame([])
for i in range(n):
    index1 = df_train2016.FIPS.values == df_graph.DST.iloc[i]
    A = df_train2016.loc[index1,:].values[0]
    df1[i] = np.concatenate([df_graph.iloc[i,:], A])
#     df_graph.iloc[i].merge(A,left_index=True, right_index=True)

df_new = df1.T
df_new.columns = ['SRC','a','b','DST','County','DEM','GOP','MedianIncome','MigraRate','BirthRate','DeathRate','BachelorRate','UnemploymentRate']
del df_new['a']
del df_new['b']

# df_new['SRC'].astype(int)
df_new.to_csv('train2016_new.csv',index=False)
print(df_new.shape)

(6176, 11)


In [13]:
df_new = pd.read_csv("train2016_new.csv", sep=',',encoding='unicode_escape',thousands=',')
df1 = df_new.groupby('SRC')[['DST','DEM','GOP']].sum().reset_index();
df2 = df_new.groupby('SRC')[['MedianIncome','MigraRate','BirthRate','DeathRate','BachelorRate','UnemploymentRate']].mean().reset_index();

del df2['SRC']
df = df1.join(df2)
df=df.rename(columns = {'SRC':'FIPS','DST':'County'})
df.to_csv('train2016_new.csv',index=False,float_format='%.3f')

print(df.shape)

(2686, 10)


### 2.3 Train the model

In [14]:
dim_input = 6
EPOCHS = 200
save_file = "best_model.pt"

best_val_acc = float("-inf")
for epoch in range(EPOCHS):
    epoch_loss = 0
    correct = 0
    loss, y_predict, val_acc, net = train("train2016_new.csv",dim_input)    
    if val_acc > best_val_acc:        
        best_val_acc = val_acc
        torch.save(net.state_dict(), save_file)
print("best validation:",best_val_acc)    
   
model = Net(dim_input);
best_val_acc_model_weights = torch.load(save_file);
model.load_state_dict(best_val_acc_model_weights);

best validation: [0.84398341]


### 2.4 test 2016 labels

In [15]:
csv_file = "test_2016_no_label.csv"
dataset = CSVDataset(csv_file, mode = 'test')
test_X = DataLoader(dataset, batch_size=64, shuffle=False)
predictions = list()
for batch_idx, input in enumerate(test_X):
    input = input.view(-1,dim_input)        
    output = model(input)
    yhat = torch.round(output)
#     yhat = output
    yhat = yhat.detach().numpy()
        
    predictions.append(yhat)
        
predictions = vstack(predictions)

In [16]:
df = pd.read_csv('sampleSubmission.csv', sep=',',encoding='unicode_escape',thousands=',')

df['Result'] = predictions.astype(int)
# df['Result'] = predictions
df.to_csv('output.csv',index=False)

In [17]:
df_graph = pd.read_csv("graph.csv", sep=',',encoding='unicode_escape',thousands=',')
df_test = pd.read_csv("test_2016_no_label.csv", sep=',',encoding='unicode_escape',thousands=',')

n,d = df_graph.shape
for i in range(n):
    index_2 = df_test.FIPS.values == df_graph.DST.iloc[i]
        
    if any(index_2): 
        df_graph.loc[i,'DST_C'] = predictions[index_2][0]
    else:
        df_graph.loc[i,'DST_C'] = np.nan

df_graph.dropna(axis = 0, how = 'any', inplace = True)

In [18]:
# def add_major(grp):
#     grp['Class_major'] = grp['DST_C'].agg(lambda x: pd.Series.mode(x)[0])
#     return grp
# class_major = df_graph.groupby('SRC').apply(add_major);

def add_major(grp):    
    if grp['SRC'].iloc[0] == grp['DST'].iloc[0]:
        a = grp['DST_C'].iloc[0]
        b = grp['DST_C'].iloc[1:].mean()
    else:
        a = grp['DST_C'].values.mean()
        b = a
        
    if (len(grp['DST_C'].values)>1):
        grp['Class_major'] = (a+b)/2
    else:
        grp['Class_major'] = a
#     grp['Class_major'] = grp['DST_C'].mean()
    return grp

class_major = df_graph.groupby('SRC').apply(add_major);
class_major['class'] = np.round(class_major.Class_major.values).astype(int)
class_major.to_csv('dis_output.csv',index=False,float_format='%.3f')

In [19]:
df = class_major.groupby('DST')['Class_major'].mean().reset_index();
df_final = pd.read_csv("output.csv", sep=',',encoding='unicode_escape',thousands=',')
n, d = df_final.shape

for j in range(n):
    index1 = df.DST.values == df_final.FIPS.iloc[j]    
    df_final.loc[j,'Result'] = df.Class_major.iloc[index1].values

df_final.Result = np.round(df_final.Result.values).astype(int)

### 2.5 Test the model using full train_2016 data

In [20]:
csv_file = "train_2016.csv"
dataset = CSVDataset(csv_file)
test_X = DataLoader(dataset, batch_size=64, shuffle=False)
predictions, actuals = list(),list()
for batch_idx, (input,target) in enumerate(test_X):
    input = input.view(-1,dim_input)        
    # validate the model
    output = model(input)
    yhat = torch.round(output)
    yhat = yhat.detach().numpy()
    
    actual = target.numpy()
    actual = actual.reshape((len(actual), 1))
        
    predictions.append(yhat)
    actuals.append(actual)
        
predictions, actuals = vstack(predictions), vstack(actuals)
acc = weighted_accuracy(predictions, actuals)
print("Final R2: ", np.round(acc,3))

Final R2:  [0.805]


### 2.6 Comments

The R2 values is around 0.8 for the model performance, which is well predicted using an ANN model built from PyTorch package. The Neural Network has 5 hidden layers, the number of nodes in the first hidden layer are increased to 160 and then are gradually decreased to 1, which makes the model more expressive. The operation from layer to layer is a linear model and the activation functions for the 3 hidden layers are ReLu while at the output layer the activation function is sigmoid, so that to obtain outputs between 0 and 1. The final output will be the rounded values of 0 or 1. The ANN model was trained using 2/3 of the training data and validated using the rest of the 1/3 data. Adam optimizer was used to update the parameters. 

When training the model, a batch size of 32 was used to make the model more dynamics. When validating the model,a batch size of 311 was used to make it smoother. 200 epoches were used and in each epoch the accuracy of the validation was calculated using the weighted_accuracy function. 